In [1]:
%load_ext autoreload
%autoreload 2

from shrimpgrad.graph import build_graph
from shrimpgrad.mlp import Neuron, Layer, MLP
from shrimpgrad.tensor import Tensor
import numpy as np

ModuleNotFoundError: No module named 'shrimpgrad.value'

In [208]:
mlp = MLP(3, [4,4,1])

In [209]:
xs = [[2.0, 3.0, -1.0],
     [3.0, -1.0, 0.5],
     [0.5, 1.0, 1.0],
     [1.0, 1.0, -1.0]]

ys = [1.0, -1.0, -1.0, 1.0]

In [238]:
def gradient_descent(xs, ys, epochs=10, lr=1e-1):
    for k in range(epochs):
        # forward pass
        ypred = [mlp(x) for x in xs]
        mse_loss = sum([(ygt - yout)**2 for ygt, yout in zip(ys, ypred)])
    
        # zero-grad
        for p in mlp.parameters():
            p.grad = 0.0
            
        # backward pass
        mse_loss.backward()
    
        # update 
        for p in mlp.parameters():
            p.data += -lr * p.grad 
    
        print(f'epoch={k}, loss={mse_loss}')
    return ypred

In [247]:
gradient_descent(xs, ys, epochs=20, lr=1.5)

epoch=0, loss=Value(data=0.00020268007130638677)
epoch=1, loss=Value(data=0.00019940024920770593)
epoch=2, loss=Value(data=0.00019622245602303985)
epoch=3, loss=Value(data=0.00019314204525431675)
epoch=4, loss=Value(data=0.0001901546470580934)
epoch=5, loss=Value(data=0.00018725614801255096)
epoch=6, loss=Value(data=0.00018444267263147506)
epoch=7, loss=Value(data=0.00018171056645196557)
epoch=8, loss=Value(data=0.00017905638054193778)
epoch=9, loss=Value(data=0.00017647685729043624)
epoch=10, loss=Value(data=0.00017396891735864511)
epoch=11, loss=Value(data=0.00017152964768272472)
epoch=12, loss=Value(data=0.00016915629043078481)
epoch=13, loss=Value(data=0.00016684623282700698)
epoch=14, loss=Value(data=0.00016459699776427594)
epoch=15, loss=Value(data=0.00016240623513536206)
epoch=16, loss=Value(data=0.0001602717138190696)
epoch=17, loss=Value(data=0.00015819131426470352)
epoch=18, loss=Value(data=0.00015616302162316202)
epoch=19, loss=Value(data=0.00015418491937848347)


[Value(data=0.9926057640527444),
 Value(data=-0.9934527671283018),
 Value(data=-0.9924868925767772),
 Value(data=0.9995559811877123)]